In [1]:
import numpy as np

class MultiClassLogisticRegressor:
    
    def fit(self, X, y, batch_size=64, lr=0.001, iter=1000, thres=0.000001, rand_seed=4): 
        np.random.seed(rand_seed) 
        self.classes = np.unique(y)
        self.thres = thres
        self.weights = self.get_randon_weights(len(self.classes), X.shape[1] + 1)
        X_train = self.add_bias(X)
        y_train = self.one_hot(y)
        self.fit_data(X_train, y_train, batch_size, lr, iter)
 
    def fit_data(self, X, y, batch_size, lr, iter): 
        while True: 
            idx = np.random.choice(X.shape[0], batch_size) 
            X_batch, y_batch = X[idx], y[idx]
            error = y_batch - self.predict(X_batch, no_bias=False)
            update = (lr * np.dot(error.T, X_batch))
            self.weights += update
            if np.abs(update).max() < self.thres: break
    
    def predict(self, X, no_bias=True):
        X = self.add_bias(X) if no_bias else X
        pre_vals = np.dot(X, self.weights.T).reshape(-1,len(self.classes))
        return self.softmax(pre_vals)
    
    def softmax(self, z):
        return np.exp(z) / np.sum(np.exp(z), axis=1).reshape(-1,1)

    def predict_classes(self, X, no_bias=True):
        self.probs_ = self.predict(X, no_bias)
        return np.argmax(self.probs_, axis=1)
  
    def add_bias(self,X):
        return np.insert(X, 0, np.ones(X.shape[0]), axis=1)
  
    def get_randon_weights(self, row, col):
        return np.zeros(shape=(row,col))
#         return np.random.random_sample(row * col).reshape(row,col)

    def one_hot(self, y):
        return np.eye(len(self.classes))[y.reshape(-1)]

In [2]:
def forward_feature_selection(X, y, n_features=2):
    results = {'best':[]}
    count = 0
    feature_list = []
    while len(feature_list) < n_features:
        results[count] = []
        for i in range(len(X[0])):
            if i not in feature_list: 
                train_model_save_result(X, y, feature_list + [i], results[count])
        best_result = get_best_result(results[count])
        feature_list = best_result['features']
        results['best'].append(best_result)
        count+= 1
    compute_best_features(results, n_features)
    return results

def backward_feature_selection(X, y, n_features=2):
    results = {'best':[]}
    count = 0
    feature_list = range(len(X[0]))
    while len(feature_list) > 1:
        results[count] = []
        for i in range(len(feature_list)):
            idx = feature_list[0:i] + feature_list[i+1:]
            train_model_save_result(X, y, idx, results[count])
        best_result = get_best_result(results[count])
        feature_list = best_result['features']
        results['best'].append(best_result)
        count+= 1
    compute_best_features(results, n_features)
    return results

def compute_best_features(results, n_features):
    best_result = None
    for result in results['best']:
        if len(result['features']) <= n_features:
            if not best_result: 
                best_result = result
            elif best_result['score'] < result['score']: 
                best_result = result
    results['best_features'] = best_result['features']
    
def train_model_save_result(X, y, idx, result_storage):
    X_ = X[:, idx]
    result = 0
    model = MultiClassLogisticRegressor()
    model.fit(X_, y, thres=1e-3)
    acc = accuracy_score(y, model.predict_classes(X_))
    result += acc
    print({'features ': idx}, {'accuracy ' :acc})
    result_storage.append({'score': result, 'features': idx})
    
def get_best_result(result_storage):
    best = None
    for model in result_storage:
        if not best: best = model
        elif model['score'] > best['score']:best = model
    return best

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data  # we only take the first two features.
Y = iris.target
backward_feature_selection(X, Y)

TypeError: unsupported operand type(s) for +: 'range' and 'range'

In [425]:
model = MultiClassLogisticRegressor()
model.fit(X[:,[3]], Y, thres=1e-3)
accuracy_score(Y, model.predict_classes(X[:,[3]]))

0.96